# Example Usage of L-GCN

## Prerequisites

In [1]:
import IPython
import pickle

import torch
import torch.nn.functional as F

from models.LGCN import LGCN


## Load Data

In [2]:
dataset = pickle.load(open("data/1hop_500", "rb"))

# normalization
val,pos = dataset.x.max(dim=0)
dataset.x /= val.abs()

In [3]:
dataset

Data(edge_attr=[1248, 2, 718], edge_attr_cutoffs=[6, 3], edge_index=[2, 1248], test_mask=[420], train_mask=[420], val_mask=[420], x=[420, 13], y=[420])

In [4]:
dataset.test_mask = dataset.test_mask.bool()
dataset.train_mask = dataset.train_mask.bool()
dataset.val_mask = dataset.val_mask.bool()

# Define Edge Module

Create the learning mechanism that is to operate on the edge populations / multi-edges.

In [5]:
class Reshape(torch.nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)


def mm_CONV(conv_channels=20, out_channels=4):
    return torch.nn.Sequential(
        torch.nn.Conv1d(2, conv_channels, kernel_size=3, stride=1, padding=1),
        torch.nn.AdaptiveMaxPool1d(1),
        torch.nn.ReLU(),
        Reshape(-1, conv_channels),
        torch.nn.Linear(conv_channels, 2*out_channels),
        torch.nn.Dropout(p=0.2),
        torch.nn.ReLU(),
        torch.nn.Linear(2*out_channels, out_channels),
        torch.nn.ReLU()
    )

# Create Example Nets

Parameter `L` determines size of latent representations of the edge populations. Parameter `H1` determines representation size in the intermediate node embedding layer. `H2` determines the size of the final output layer and should agree with the downstream task configured in the data set.

In the GCN layers, the following controls are available:
* `make_bidirectional` offers bidirectional propagation over directed graphs
* `neighbor_nl` offers additional per-neighbor nonlinearity *inside* the graph convolution (L-GCN+)
* `DVE` provides the option of embedding local neighborhood aggregations of the latent representations (mean-pool) directly on the nodes, before proceeding with the GCN (L-GCN+ & DVE)


In these examples, edge populations are pre-padded with zeros and sorted by original sequence length, accompanied by batch cut-offs for faster processing. The `edge_attr_cutoffs` parameter may be omitted to proceed without batching.


### L4-GCN (bidirectional propagation)

In [6]:
class LGCN_Net(torch.nn.Module):
    def __init__(self, L=4, H1=20, H2=2):
        super().__init__()
        self.conv1 = LGCN(dataset.num_features, H1, mm_CONV(out_channels=L), L=L,
                          make_bidirectional=True)
        self.conv2 = LGCN(H1, H2, mm_CONV(out_channels=L), L=L,
                          make_bidirectional=True)

    def forward(self, data):
        x, edge_index, edge_attr, edge_attr_cutoffs = data.x, data.edge_index, data.edge_attr, data.edge_attr_cutoffs
        x = self.conv1(x, edge_index, edge_attr, edge_attr_cutoffs=edge_attr_cutoffs)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr, edge_attr_cutoffs=edge_attr_cutoffs)

        return F.log_softmax(x, dim=1)

### L4-GCN+ (bidirectional propagation)

In [7]:
class LGCN_Net2(torch.nn.Module):
    def __init__(self, L=4, H1=20, H2=2):
        super().__init__()
        self.conv1 = LGCN(dataset.num_features, H1, mm_CONV(out_channels=L),L=L,
                          make_bidirectional=True,
                          neighbor_nl=True)
        self.conv2 = LGCN(H1, H2, mm_CONV(out_channels=L), L=L,
                          make_bidirectional=True,
                          neighbor_nl=True)

    def forward(self, data):
        x, edge_index, edge_attr, edge_attr_cutoffs = data.x, data.edge_index, data.edge_attr, data.edge_attr_cutoffs
        x = self.conv1(x, edge_index, edge_attr, edge_attr_cutoffs=edge_attr_cutoffs)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr, edge_attr_cutoffs=edge_attr_cutoffs)

        return F.log_softmax(x, dim=1)

### L4-GCN+ & DVE (bidirectional propagation)

In [8]:
class LGCN_Net3(torch.nn.Module):
    def __init__(self, L=4, H1=20, H2=2):
        super().__init__()
        self.conv1 = LGCN(dataset.num_features, H1, mm_CONV(out_channels=L), L=L,
                          make_bidirectional=True,
                          neighbor_nl=True,
                          DVE=True)
        self.conv2 = LGCN(H1, H2, mm_CONV(out_channels=L), L=L,
                          make_bidirectional=True,
                          neighbor_nl=True)

    def forward(self, data):
        x, edge_index, edge_attr, edge_attr_cutoffs = data.x, data.edge_index, data.edge_attr, data.edge_attr_cutoffs
        x = self.conv1(x, edge_index, edge_attr, edge_attr_cutoffs=edge_attr_cutoffs)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_attr, edge_attr_cutoffs=edge_attr_cutoffs)

        return F.log_softmax(x, dim=1)

# Example Training

In [9]:
epochs = 100
lr = 5e-4
weight_decay = 5e-4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LGCN_Net3().to(device)
data = dataset.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
model.train()

train_class_ratio = dataset.y[dataset.train_mask].sum().item()/dataset.y[dataset.train_mask].shape[0]
train_class_weights = torch.Tensor([train_class_ratio,1-train_class_ratio]).to(device)

out = display(IPython.display.Pretty('Starting'), display_id=True)
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = F.nll_loss(model(data)[data.train_mask], data.y[data.train_mask], weight=train_class_weights)
    loss.backward()
    optimizer.step()    
    out.update(IPython.display.Pretty(f"Epoch {epoch+1}/{epochs}"))

out.update(IPython.display.Pretty("Done."))
model.eval()

test_acc = model(data).max(dim=1)[1][data.test_mask].eq(data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
print('Testing Accuracy: {:.4f}'.format(test_acc))
print('Number of parameters: {}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))


Done.

Testing Accuracy: 0.7613
Number of parameters: 8930
